# Packages Laden

In [2]:
from igraph import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Willkommen zum vierten Python-Problem Set in diesem Modul!  
Sie finden hier zwei Aufgaben (*Exercise 7* und *Exercise 8*) zu Inhalten aus Kapitel 3 *Social Network Analysis* - konkret zu den Teilen 3.3 *Communities in Netzwerken* und 3.4 *Information Diffusion in sozialen Netzwerken*.

# Exercise 7 -- Communities in Netzwerken

Rufen Sie sich das Netzwerk *karate* zurück in Erinnerung, indem Sie den folgenden Code ausführen.

### Aufgabe 7.1

In [3]:
# {"7_1"}
# Import des Karate Netzwerks
karate = pd.read_csv("../PS_Nr_2_Python/data/Karate/KARATE.csv", sep=";", header=None, names=['A','B'])

In [4]:
def create_edge_list(node_list1, node_list2):
    edge_list = []
    for node_1, node_2 in zip(node_list1, node_list2):
        edge = (node_1, node_2)
        edge_list.append(edge)
    return edge_list

In [5]:
karate_edges = create_edge_list(karate['A'].to_list(), karate['B'].to_list())

In [ ]:
# Erzeugung des Karate Graphs
graph_karate = Graph(edges=karate_edges, directed=True)

# Plotten des Graphen karate
plot(graph_karate)

Wie Sie sehen können, sind die Knoten des Graphen - die Teilnehmer des Karateclubs - in zwei verschiedenen Farben dargestellt. Sie stellen die beiden Untergruppen dar, in die der Club nach seiner Aufspaltung zerfallen ist (igraphdata, 2015). Die Rede ist also von sogenannten **Communities**.  
Im Skript haben Sie bereits einige Methoden zur Community Detection kennengelernt. In dieser Aufgabe wollen wir das *'igraph'*-Package nutzen, um Communities in Graphen ausfindig zu machen. Darüber hinaus werden wir die verschiedenen Algorithmen miteinander vergleichen.

Luke (2015) merkt an, dass das Package *'igraph'* hinsichtlich Community Detection den Großteil der relevanten Algorithmen anbietet und zeigt eine Übersicht über die verschiedenen Algorithmen und ihre Umsetzung in R.

Im Skript werden verschiedene Arten von Algorithmen angesprochen, von denen wir uns den folgenden näher widmen möchten:  
* Divisive Algorithmen 
* Agglomerative Algorithmen
* Maximierung von Zielkriterien

Zu den genannten Kategorien werden wir je ein Beispiel untersuchen.

## a) Divisive Algorithmen - Edge-Betweenness-Algorithmus

Ein Beispiel eines divisiven Algorithmus ist der **Edge-Betweenness-Algorithmus**. Er ist in R in der Funktion *cluster_edge_betweenness(graph, weights)* implementiert. Die Idee und die darauf basierende Umsetzung wird in igraphdata (2015) folgendermaßen beschrieben:

Die Edge-Betweenness gibt an, auf wie vielen kürzesten Pfaden eine Kante sitzt. Gegeben ein Graph besteht aus mehreren Communities, so verlaufen durch diejenigen Kanten, die die Communities verbinden, sehr viele kürzeste Pfade. Umgekehrt ist die Wahrscheinlichkeit sehr hoch, dass eine Kante zwei Communities trennt, wenn sie eine hohe Edge-Betweenness besitzt. Im Sinne eines divisiven Algorithmus werden daher die Kanten mit der höchsten Edge-Betweenness nach und nach entfernt, um ein hierarchisches Cluster zu bilden.
- - -
*Aufgabe*: Wenden Sie den Befehl *cluster_edge_betweenness(graph, weights)* auf den Graphen *karate* an. Das Argument *weights* verwendet dabei automatisch das Kantenattribut *weight* des Graphen *karate*, falls nicht *weights=NULL* gesetzt wird. Um konsistent mit den bisherigen Betrachtungen zu sein, werden wir hier die Gewichte ebenfalls nicht berücksichtigen.

### Aufgabe 7.2

In [7]:
# {"7_2"}
#Wenden Sie den oben beschriebenen Befehl an und speichern Sie das
#Ergebnis in der Variable karate_cl_edgebtw
karate_cl_edgebtw = graph_karate.community_edge_betweenness()

Grundsätzlich liefern fast alle Clusterbefehle in *'igraph'* ein Objekt der Klasse *communities* zurück. Hierauf können unter anderem die folgenden Befehle angewandt werden (Kolaczyk und Csárdi, 2014; Kapitel 4.4.1):  
* *length(communities)* gibt die Anzahl der Communities im Graphen an 
* *sizes(communities)* gibt die Anzahl der Knoten in jeder Community an 
* *membership(communities)* gibt für jeden Knoten die Community an, zu der er gehört 
* *plot(communities, graph)* stellt die Communities grafisch dar, indem er den Graphen mit farbig markierten Communities plottet 
* *dendPlot(communities)* stellt das hierarchische Cluster in einem Dendrogramm dar (Näheres dazu später)

Der letzte Befehl (*dendPlot*) ist leider nicht auf den Output jedes Clusterbefehls anwendbar - Voraussetzung hierfür ist, dass ein hierarchisches Cluster zurückgegeben wurde.

- - -
*Aufgabe*: Wenden Sie nun alle Befehle auf das Communities-Objekt *karate_cl_edgebtw* an.
- - -

### Aufgabe 7.3

In [8]:
# {"7_3"}
#Aus wie vielen Communities besteht karate?
print(len(karate_cl_edgebtw))
#Wie groß sind die einzelnen Communities?
print(karate_cl_edgebtw.sizes())
#Lassen Sie sich für die ersten sechs Knoten anzeigen, 
#zu welcher Community sie gehören
#Tipp: Befehl head (per default Ausgabe von sechs Zeilen)
karate_cl_edgebtw.communities[0:5]
#Plotten Sie den Graphen mit hervorgehobenen Communities
#Achten Sie hierbei darauf, dass die Funktion plot(x,y)
#als Eingabeparameter für x das Communities-Objekt und
#und für y das Graph-Objekt erhält
plot(x=karate_cl_edgebtw, y=karate)
#Plotten Sie das Dendrogramm des Clusters.
#Verwenden Sie auch hier für die Funktion dendPlot(x)
#das Communities-Objekt für x.
dendPlot(x=karate_cl_edgebtw)

TypeError: object of type 'VertexDendrogram' has no len()

Wie Sie sehen, stellt das Dendrogramm alle möglichen Ebenen des Clustering dar: Von der Unterteilung des Graphen in seine einzelnen Knoten als separate Communities (ganz unten) bis hin zur Betrachtung des Graphen als eine einzige Community (oberste Linie).

Die Communities, die der Edge-Betweenness-Algorithmus ermittelt hat, weichen leider ab von denen, welche im Graphen *karate* als Knotenattribut *Faction* angegeben und farblich im ersten Plot dieser Aufgabe dargestellt sind. Testen wir im Folgenden, was andere Algorithmen ergeben.

## b) Agglomerative Algorithmen - Label-Propagation-Algorithmus

Im Skript haben Sie bereits ein Beispiel für einen agglomerativen Algorithmus kennengelernt: den **Label-Propagation-Algorihmus**.  
In *'igraph'* wird dieser im Befehl *cluster_label_prop(graph, weights, initial, fixed)* umgesetzt. Dabei können Gewichte verwendet werden und es kann auch ein Cluster definiert werden, welches zu Beginn des Algorithmus genutzt wird (*initial*). Das Argument *fixed* legt für diesen Fall die Knoten fest, deren Zuordnung im Laufe des Algorithmus nicht verändert werden darf. Diese letzten beiden Argumente werden wir im Folgenden jedoch nicht verwenden.

- - -
*Aufgabe*: Nutzen Sie im folgenden Code-Chunk den Label-Propagation-Algorithmus, um die Communities des Graphen *karate* zu ermitteln (ohne Berücksichtigung der Gewichte, d.h. *weights=NA*).
- - -